# Distributional assumption -- The distribution includes biases!

Word vectors encode the linguistic contexts in which words appear. We can therefore ask how much linguistic bias -- which echoes societal biases -- is encoded in word vectors. Because much of web text is written by white, heterosexual North American men, as well as their Western European counterparts, the texts include biases that get encoded via distributional semantics methods.

Societal biases include, but are not exclusive to

* race
* ethnicity
* gender
* sexuality
* disabled status
* age
* intersections of all of these

We know that neural language models produce text that amplifies these biases. For example, recent studies suggest that neural language models and chat systems like ChatGPT have a predominantly "cool (white, affluent) mom" orientation toward social problems. The models in production today have lots of bumpers to prevent producing racist, sexist, etc. text.

For this, I will show you a cool projection method called "subspace" analysis, which produces a vector between two end-points.

One of the first places studying gender bias in word vectors was done was in [Bolukbası et al. (2016)](https://proceedings.neurips.cc/paper_files/paper/2016/file/a486cd07e4ac3d270571622f4f316ec5-Paper.pdf) who used a combination of:

* Static word vectors from `word2vec`
* Difference vectors between two "endpoints" of a binary spectrum (masculine <--> feminine)
* Principal components analysis over that endpoint
* The above steps produce a "gender subspace" vector
* Compute similarity between each vector for an occupation -- which can be biased toward different genders to different degrees, and verified with census data -- and the gender subspace vector

For our experiment, we will load in some vectors from `spaCy` and quantify the degree to which gender bias exists for some specific words:

```python
pairs = [
  ('he', 'she'),
  ('his', 'hers'),
  ('him', 'her'),
  ('his', 'her'),
  ('John', 'Mary'),
  ('himself', 'herself'),
  ('father', 'mother'),
  ('guy', 'gal'),
  ('boy', 'girl'),
  ('male', 'female')
]
```

### Food for thought
* Can you think of other pairs of words that might be useful here? (brother, sister), (wife, husband), (doctor, nurse) ?
* What about other dimensions from one end to another might be able to use this method? age - young <--> old, first name last name
* How can these "endpoints" be used for answering questions other than bias? maybe it can look into 

In [ ]:
# imports
from sklearn.preprocessing import normalize
import pandas as pd
import numpy as np

import spacy
! python -m spacy download en_core_web_md

nlp = spacy.load('en_core_web_md')

from sklearn.decomposition import TruncatedSVD # to project down into smaller dimension
from scipy.spatial.distance import cosine

from plotnine import *

## Get occupations

There are lots of ways to get career or occupational bias. Here, we download a dataset that has lots of different biases -- gender, race, ethnicity -- and was used to study bias in GPT-2. We can check that the Bolukbası method correlates with:

- The gender bias in US Census data
- The gender bias that GPT-2 produces

https://github.com/oxai/intersectional_gpt2

Data:
https://github.com/oxai/intersectional_gpt2/blob/master/data/GPT-2/US_data/us_rows_data.csv

In [ ]:
# upload the file
df = pd.read_csv("documents/gpt_vs_us_data.csv")
df.head()

# us_white_F is average bias in the census for white women
# gpt_white_F is the average bias produced by GPT-2
# ask dr jacobs what this means ~~~~~~~~~

In [ ]:
def extract_row(frame, field): # extract all data from a specific row in frame
    result = []
    for i, row in frame.iterrows():
      row_val = row[field]
      val_split = row_val.split("/") # some data entries have multiple names separated by a "/"
      for vals in val_split:
        result.append(vals.strip()) # append to result array
    return result

In [ ]:
jobs = extract_row(df, "job")

## Define the words used for the "endpoints" of the projection

The idea here is that if we compute the "difference" (subtraction) between "masculine" vectors and "feminine" vectors, we can compute the subspace that corresponds to the axes that are correlated with some linguistic dimension of bias.

In [ ]:
pairs = [
  ('he', 'she'),
  ('his', 'hers'),
  ('him', 'her'),
  ('his', 'her'),
  ('John', 'Mary'),
  ('himself', 'herself'),
  ('father', 'mother'),
  ('guy', 'gal'),
  ('boy', 'girl'),
  ('male', 'female'),
  ('masculine', 'feminine'),
]

In [ ]:
def create_endpoints(model, pairs): # get difference vectors between pairs, create endpoints
    diff_vectors = [normalize(model(x).vector.reshape(1, -1) - model(y).vector.reshape(1, -1))[0] for x, y in pairs] #model(x) gets word embeddings from model
    return diff_vectors

In [ ]:
diff_vectors = create_endpoints(nlp, pairs)
diff_vectors[0].shape

# Learn the subspace using PCA

PCA learns the dominant dimensions that characterize a space. If we project to a single dimension, this will result in one "component" that characterizes the differences between "masculine" vectors and "feminine" vectors.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f5/GaussianScatterPCA.svg/1920px-GaussianScatterPCA.svg.png" width=400>

PCA demonstration, Wikipeda

In [ ]:
pca = TruncatedSVD(n_components=1) # one gender dimension
pca.fit(diff_vectors)

As an annoying implementational detail, Bolukbası normalize word vectors to the unit circle -- but this might not be necessary.

In [ ]:
def compute_uc_vectors(model, vectors): # get unit circle vectors
    uc_vectors = [normalize(model(vector).vector.reshape(1, -1))[0] for vector in vectors]
    return uc_vectors

In [ ]:
job_vectors = compute_uc_vectors(nlp, jobs)
job_vectors[0].shape

# Compute the bias scores using cosine similarity

We can look at how much the learned axis above "weighs" in the same direction as any particular job word vector. Above, we extract the vectors for a bunch of job titles, and we'll compute the cosine similarity (angle difference) between the "gender subspace" vector and the job title vector.

The numbers weigh on the differences -- we subtracted feminine from masculine, so higher values indicate more feminine "loadings" and lower values are more "masculine."

In [ ]:
def compute_bias(extracted, vectors): # get bias scores from df data, and created masculine/feminine subspace
    bias_scores = []
    for i, vector in enumerate(vectors):
        vector_data = extracted[i]
    bias_scores.append((vector_data, cosine(vector, normalize(pca.components_).reshape(-1)))) # make this its own function
    return bias_scores

In [ ]:
scores = pca.transform(job_vectors).flatten().tolist()

In [ ]:
scores_list = list(zip(scores, jobs))
scores_list

In [ ]:
scores_df = pd.DataFrame(scores_list, columns=['bias', 'job']) # create our own df from subsapce and jobs data
scores_df

### Correlation of spaCy vectors to GPT-2

In [ ]:
def correlation_plot(df_one, df_two, x_axis, y_axis, label, x_axisName, y_axisName):
    plot = (ggplot(df_one.merge(df_two),
       aes(x=x_axis, y=y_axis, label=label)) + geom_label() + theme_bw()
       + xlab(x_axisName)
       + ylab(y_axisName))
    return plot.draw()

In [ ]:
scores_df.merge(df)

In [ ]:
correlation_plot(scores_df, df, "bias", "gpt_white_F", "job", "spaCy vector bias, Bolukbasi method", "GPT vector bias")

### Correlation of spaCy vectors to US Census data

In [ ]:
correlation_plot(scores_df, df, "bias", "us_white_F", "job", "spaCy vector bias, Bolukbasi method", "US census bias")

In [ ]:
"""
1. Extracted dataset that studied GPT-2 bias
2. Define endpoints for projection using spacy model. We used vectors that were pairs of opposite sides to each other. (masculine, feminine). We subtracted
them. This produces difference vectors that we use as endpoints to project a subspace, which includes one "component" that characterizes the difference between
masculine and feminine vectors. Now we have a subspace to utilize that has masculine and feminine endpoints, as well as one component that has the differences.
3. We now compute the bias scores using cosine similarity - angle difference (the component). 
4. We compute the cosine similarity between the gender subspace and the job title vector.
5. The numbers weigh on the differences -- we subtracted feminine from masculine, so higher values indicate more feminine "loadings" and lower values are more "masculine."
6. Using the bias score vectors, where it is (job title, bias score), we create a new dataframe to store these vectors.
7. We then use this dataframe to create a correlation graph between the biases seen in the study and our bias scores.

"""

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
mBERT = BertModel.from_pretrained('bert-base-multilingual-cased')

In [ ]:
pairs_japanese = [
    ('彼', '彼女'),  # he, she or boyfriend, girlfriend
    ('彼の', '彼女の'),  # his, her
    ('男性', '女性'),  # male, female
    ('男の子', '女の子'),  # boy, girl
    ('父', '母'),  # father, mother
]

pairs_spanish =  [
    ('él', 'ella'),  # he, she
    ('su', 'su'),     # his, her (Note: Spanish has the same word for his and her)
    ('hombre', 'mujer'),  # man, woman
    ('padre', 'madre'),   # father, mother
    ('niño', 'niña'),     # boy, girl
    ('actor', 'actriz'),  # actor (male), actress (female)
    ('rey', 'reina'),     # king, queen
    ('hermano', 'hermana')
]
# brother, sister

In [ ]:
# get word embeddings from mBERT and create endpoints
def mBERT_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = mBERT(**inputs)
    # Take the mean of the embeddings across the token sequence to get a single embedding vector
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy().reshape(1,-1)
    
def mBERT_endpoints(vectors): # get difference vectors between pairs, create endpoints
    diff_vectors = [normalize(mBERT_embeddings(x).reshape(1, -1) - mBERT_embeddings(y).reshape(1, -1))[0] for x, y in vectors] #model(x) gets word embeddings from model
    return diff_vectors

In [ ]:
mBERT_vectors = mBERT_endpoints(pairs_japanese)
mBERT_vectors # create endpoints for Japanese masculine<->feminine words to use for subspace

In [ ]:
m_pca = TruncatedSVD(n_components=1) # one gender dimension
m_pca.fit(mBERT_vectors)
#CONFUSED!!

In [ ]:
def compute_mBERT_vectors(model, vectors): # get unit circle vectors
    uc_vectors = [normalize(mBERT_embeddings(word)).flatten() for word in vectors]
    return uc_vectors

In [ ]:
m_vectors = compute_mBERT_vectors(mBERT, jobs)
m_vectors # get number representations of jobs data with mBERT model

In [ ]:
# This projects the job description vectors onto the gender subspace created by the PCA. 
# meaning each job vector is now represented by how much it aligns with the gender dimension derived from the Japanese word pairs
mBERT_list = m_pca.transform(m_vectors).flatten() 

# pairs each projected value - the new value after pca transform 
# (indicating the job's alignment with the gender dimension) with its corresponding job description
mBERT_zip = list(zip(mBERT_list, jobs))
mBERT_zip

In [ ]:
mBERT_df = pd.DataFrame(mBERT_zip, columns = ["bias", "job"]) # create jobs data frame
mBERT_df

In [ ]:
# Higher is more feminine, Lower is more masculine
correlation_plot(mBERT_df, df, "bias", "gpt_white_F", "job", "spaCy vector bias, Bolukbasi method", "GPT vector bias")

In [ ]:
mBERT_spanish = mBERT_endpoints(pairs_spanish)
mBERT_spanishPca = TruncatedSVD(n_components = 1)
mBERT_spanishPca.fit(mBERT_spanish)

mBERT_spanish_jobs = compute_mBERT_vectors(mBERT, jobs)
mBERT_spanish_list = list(zip(mBERT_spanishPca.transform(mBERT_spanish_jobs).flatten(), jobs))
mBERT_spanish_list

In [ ]:
mBERT_spanish_df = pd.DataFrame(mBERT_spanish_list, columns = ["bias", "job"])
mBERT_spanish_df

In [ ]:
# Higher is more feminine, Lower is more masculine
correlation_plot(mBERT_spanish_df, df, "bias", "gpt_white_F", "job", "spaCy vector bias, Bolukbasi method", "GPT vector bias")

In [ ]:
# TODO: JAPANESE JOBS
pairs_japanese_jobs = 